In [ ]:
import os
import sys
import csv
import time
import ast
import json
import pandas as pd
from pandas.io.json import json_normalize
import elasticsearch #version (6, 3, 1)
from elasticsearch import helpers
from elasticsearch import Elasticsearch

In [ ]:
# make sure no encode issue
#reload(sys)
#sys.setdefaultencoding('utf8')

currentDirectory = os.getcwd()
dataFilePath = currentDirectory


In [ ]:
dataFilePath

In [ ]:
class ElasticSearchExporter(object):
    def export(self, host, indexDbName, docType, queryString, UserName, Password,
               form='csv', queryJson=None, size=100, fields="all", start=None, end=None):

        es = Elasticsearch(host, http_auth=(UserName, Password),
                           scheme="https",
                           verify_certs=False,
                           timeout=1000)

        # define query
        query = None
        if queryJson:
            query = json.loads(queryJson)
        else:
            query = dict(
                query=dict(
                    query_string=dict(
                        query=queryString
                    )
                )
            )
            # query["sort"] = [{"time":"desc"}]
            query["size"] = size

        map_index_reference = list(es.indices.get(indexDbName))
        map_index = map_index_reference[0]
        map_index_reference = [i for i in map_index_reference if start <=i[-len(start):] and end >= i[-len(end):]]


        if start == None:
            if end == None:
                index_check = map_index_reference
            else:
                index_check = map_index_reference

        elif end == None:
            index_check = map_index_reference

        else:
            start_position = map_index_reference.index(map_index_reference[0])
            end_position = map_index_reference.index(map_index_reference[-1]) + 1
            index_check = map_index_reference[start_position: end_position]
            print(f'list of indexes mapped : {index_check}')


        # Set handler to elasticsearch
        scanResponse = helpers.scan(client=es, query=query, scroll="10m", index=indexDbName, size=size, doc_type=docType, clear_scroll=False, request_timeout=3000)

        counter = 0
        # form = outputFiles.split(".",1)[1]

        json_file = []
        for row in scanResponse:
            #if counter <= 10000:
               
            if row['_index'] in index_check:
                if fields == "all":
                    json_file.append(row)

                else:
                    json_file.append(row['_source'][fields])

                counter += 1

            else:
                counter = counter
                    
            #else:
             #   break

        if form == 'df':
            data_frame = pd.DataFrame(json_file)

            #for c in data_frame.columns:
                #data_frame[c] = data_frame[c].map(lambda i : ast.literal_eval(i))

            col = list(data_frame.columns)

            d = []
            for c in col:
                if type(data_frame.iloc[1][c]) == dict:
                    d.append(c)

            for i in d:
                df1 = json_normalize(data_frame[i])
                data_frame = pd.concat([data_frame, df1], axis=1, join_axes=[df1.index])

            data_frame.drop(d, axis=1, inplace=True)


            print('%s lines was expotred' % (counter))
            return data_frame


        if form == 'json':
            print('%s lines was expotred' % (counter))
            return json_file

In [ ]:
exporter = ElasticSearchExporter()
exporter.export(form = '<format>', #df or json
                host = '<hostname>', #e.g. : 'https://msb-elk-int.de050.corpintra.net:9200'
                UserName = '<user_name>',
                Password = '<password>',
                indexDbName ='<indexname>', #e.g. : 'it-ocs_msb_prd-tsb-te1500ac-process-v1-2019.01.*'
                start = '<index_start>', #e.g. : '15' or '01.15' for moth and date
                end = '<index_end>', #e.g. : '16' or '01.16' for month and date
                docType = '<doc_type>', #e.g. : 'PROCESS'
                queryString = None,
                queryJson = '<query>', #'{"query": {"bool": {"should": [{"match": {"Payload.CYCLETIME.RobotName":"UB64_150RB_100"}},{"match": {"Payload.CYCLETIME.RobotName":"UB64_150RB_200"}}]}}}
                size = 1000,
                fields = "all") 